In [9]:
# Dependencies and Setup
import hvplot.pandas
import geoviews as gv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import gmaps
import os 
import json

In [12]:
# Import API key
from api_keys import geoapify_key

# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
cities= "../cities.csv"
city_data_df = pd.read_csv('cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
mat_plt_1= city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo= True, 
    tiles= 'OSM',
    frame_width= 700,
    frame_height= 500,
    color= 'City',
    size= 'Humidity',)


# Display the map
mat_plt_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_df= city_data_df.loc[(city_data_df['Max Temp']<28) & (city_data_df['Max Temp']>22) &
(city_data_df['Wind Speed']<4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
narrow_df = narrow_df.dropna()

# Display sample data
narrow_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
45,45,kapaa,22.0752,-159.3190,22.99,84,0,3.60,US,1666108257
51,51,hilo,19.7297,-155.0900,26.27,83,0,2.57,US,1666108260
63,63,banda,25.4833,80.3333,24.62,52,0,2.68,IN,1666108268
152,152,kahului,20.8947,-156.4700,23.80,60,0,3.09,US,1666108246
197,197,gat,31.6100,34.7642,24.38,100,0,3.69,IL,1666108356


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= narrow_df[['City', 'Country', 'Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,
51,hilo,US,19.7297,-155.0900,83,
63,banda,IN,25.4833,80.3333,52,
152,kahului,US,20.8947,-156.4700,60,
197,gat,IL,31.6100,34.7642,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [16]:
import requests

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Set parameters to search for a hotel
    categories = "accommodation.hotel"
    limit = 1
    radius = 10000
    params = {
        "categories": categories,
        "radius": radius,
        "limit": limit,
        "bias": f"proximity:{longitude},{latitude}",
        "apiKey": geoapify_key
    }

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Checking if the request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        data = response.json()
        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_name = data["features"][0]["properties"]["name"]
        except (KeyError, IndexError):
            hotel_name = "No hotel found"
    else:
        hotel_name = "API request failed"
    
    # Store the hotel name in the DataFrame
    hotel_df.loc[index, "Hotel Name"] = hotel_name
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_name}")

# Display sample data
hotel_df.head()

Starting hotel search
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: מלון גולדן טאוור
tikaitnagar - nearest hotel: Maa patesawri sweet hous
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: Shivpuri Hotel
roebourne - nearest hotel: Ibis Styles Karratha
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
lakki marwat - nearest hotel: Madina Hotel
guerrero negro - nearest hotel: Plaza sal paraiso
pali - nearest hotel: Hotel Pawan International


,City,Country,Lat,Lng,Humidity,Hotel Name
45,kapaa,US,22.0752,-159.3190,84,Pono Kai Resort
51,hilo,US,19.7297,-155.0900,83,Dolphin Bay Hotel
63,banda,IN,25.4833,80.3333,52,#acnindiafy21
152,kahului,US,20.8947,-156.4700,60,Maui Seaside Hotel
197,gat,IL,31.6100,34.7642,100,מלון גולדן טאוור


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng','Lat', 
    geo=True, 
    tiles= 'OSM',
    frame_width= 700,
    frame_height= 500,
    color= 'City',
    hover_cols=['Hotel Name','Country']
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)